In [2]:
import sys
sys.path.append(rf"/home/a/Lwy_research")
from Functions import *

In [3]:
# 计算完整的秒序列
def calc_sec_list(keep_last = False):
    time_series=[]  #所有的秒序列,不包括早尾盘集合竞价(也就是093000-145659的)
    for i in [9,10,11,13,14]:
        for j in range(60):
            if i==9 and j<30:
                continue
            elif i==11 and j>=30:
                continue
            elif i==14 and j>=57:
                continue
            time_series.extend([int(str(i)+'0'*(2-len(str(j)))+str(j)+'0'*(2-len(str(x)))+str(x)) for x in range(60)])
    if keep_last: # 保留最后一个时点
        time_series += [150000]
    return time_series

# 转化为1秒级时间,比如如果time为93524380，且转化的1s时间为93524000
def get_second_1s(time_10ms):
    div = 1000
    return round(round(time_10ms // div) * div) # 先取整再标准化为Level2时间


In [29]:
# 计算单只股票的lob数据
def calc_lob_data_by_code(code, trans_data, order_data, get_second, second_list):
    try:
        ORDERBOOK = orderbook(code)
        td_full = pd.concat([order_data, trans_data]).sort_values(['time','order_or_trans'])  #时间戳相同时先算委托再算成交
        td_full['Volume']=td_full['Volume'].abs() # 所有撤单数据也标记为volume
        del td_full['order_or_trans']


        # 将订单时间分为以下三部分
        td_call_actual = td_full[td_full["time"] < 92500000]
        td_925 = td_full[(td_full["time"] >= 92500000)&(td_full['time'] < 93000000)]
        td_full = td_full[(td_full["time"] >= 93000000) & (td_full["time"] <= 150100000)]

        td_call_actual2 = td_call_actual.values
        td_9252 = td_925.values
        td_full2 = td_full.values

        final_result = pd.DataFrame(index=[92500000]+ get_second(td_full["time"]).unique().tolist(),
                                    columns=["code"] + ["bp_{}".format(x) for x in range(1,21)]+\
                                    ["bv_{}".format(x) for x in range(1,21)]+\
                                    ["bn_{}".format(x) for x in range(1,21)]+\
                                    ["sp_{}".format(x) for x in range(1,21)]+\
                                    ["sv_{}".format(x) for x in range(1,21)]+\
                                    ["sn_{}".format(x) for x in range(1,21)]+["price", "vol", "amt", "num"])  # 买卖20档盘口，包含档口价、档口挂弹量和笔数

        final_result["code"] = code

        final_result_list = []
        #集合竞价过程的撮合
        for i in range(td_call_actual2.shape[0]):
            data0 = td_call_actual2[i]
            ORDERBOOK.deal_data(data0)

        # 9:25确定开盘价
        vol = 0
        amt = 0
        num = 0
        for i in range(td_9252.shape[0]):
            data0 = td_9252[i]
            OBACK = ORDERBOOK.deal_data(data0, is_925=True)
            vol += OBACK[0]
            amt += OBACK[1]
            num += OBACK[2]
        if vol > 0:
            ORDERBOOK.revise_trade()
        tot_list = ORDERBOOK.get_orderbook_slice()
        tot_list = tot_list + [ORDERBOOK.price_now, vol, amt, num]
        final_result_list.append(tot_list)

        # 以上步骤实盘过程中应该在9:30之前一次性完成
        # 9:30后连续竞价随着实盘一接收到订单就开始计算，流式完成计算和处理
        vol = 0
        amt = 0
        num = 0
        last_timestamp = None
        for i in range(td_full2.shape[0]):
            data0 = td_full2[i] # 100 ms
            now_timestamp = get_second(data0[1])
            # 如果freq=all，则只要时间戳变动就必须记录，否则就根据频率记录
            if last_timestamp != None and now_timestamp != last_timestamp:
                tot_list = ORDERBOOK.get_orderbook_slice()
                tot_list = tot_list + [ORDERBOOK.price_now, vol, amt, num]
                final_result_list.append(tot_list)
                vol = 0
                amt = 0
                num = 0
            OBACK = ORDERBOOK.deal_data(data0)
            vol += OBACK[0]
            amt += OBACK[1]
            num += OBACK[2]
            last_timestamp = now_timestamp
            if i == td_full2.shape[0]-1:
                tot_list = ORDERBOOK.get_orderbook_slice()
                tot_list = tot_list + [ORDERBOOK.price_now, vol, amt, num]
                final_result_list.append(tot_list)
        final_result[final_result.columns.tolist()[(final_result.columns.tolist().index("bp_1")):]] = np.vstack(final_result_list)
        
        out_fea=['bp_1','sp_1','bv_1','sv_1','price','vol','amt'] # 这里outfea有可能后面会变
        tick_lobdata_used = final_result.loc[:,out_fea].reindex(second_list)
        tick_lobdata_used[['bp_1','sp_1','bv_1','sv_1','price']]=tick_lobdata_used[['bp_1','sp_1','bv_1','sv_1','price']].fillna(method='ffill') #价格数据向后填充
        tick_lobdata_used[['vol','amt']]=tick_lobdata_used[['vol','amt']].fillna(0)
        tick_lobdata_used["Code"] = code
        tick_lobdata_used = tick_lobdata_used.reset_index()
    except Exception as e:
        tick_lobdata_used = pd.DataFrame()
        print(code, e)
    return code, tick_lobdata_used

In [49]:
date = "20240913"
order_data = pd.read_feather(rf"./example_data/sample_order_{date}.fea").rename(columns={'orderId':"id", 'orderPrice':'Price','orderVolume':'Volume','orderSide':'Side'})
# order数据中，如果属于撤单则order_or_trans标记为1，反之标记为0
order_data['order_or_trans'] = order_data['orderType'].isin(['D','C']) * 1
order_data=order_data[['code','time','Price','Volume','Side','orderType','orderIdOrigin','order_or_trans']]

trans_data = pd.read_feather(rf"./example_data/sample_trans_{date}.fea").rename(columns={'transId':"id", 'transPrice':'Price','transVolume':'Volume','transSide':'Side'})
trans_data['order_or_trans']=1  # 逐笔成交数据order_or_trans均标记为1
trans_data=trans_data[['code','time','Price','Volume','Side','bidId','askId','order_or_trans']]

code_list = list(order_data['code'].unique())

# 将数据转化为dict，加快运算
order_data_dict = change_df2dict(order_data)
trans_data_dict = change_df2dict(trans_data)

freq = "1s"
get_second = eval(f"get_second_{freq}") # 看看需要几秒频的,如果freq = "all"则默认全部保存
second_list = calc_sec_list(keep_last=True)
second_list = [x*1000 for x in second_list]

sec_lobdata = Parallel(n_jobs=32)(delayed(calc_lob_data_by_code)(code, trans_data_dict[code], order_data_dict[code], get_second, second_list) for code in tqdm(code_list[:]))
sec_lobdata_df = pd.concat([x[1] for x in sec_lobdata])
sec_lobdata_df.reset_index(drop = True).to_feather(rf"./example_data/sample_lob_{date}.fea")

100%|██████████| 1000/1000 [00:46<00:00, 21.38it/s]
